In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as grd
import itertools as it
import re
import pickle
from collections import defaultdict
import itertools
import pandas as pd

if not os.getcwd().endswith("spacetimeproject"):
    os.chdir("..")
    print(os.getcwd())
from mpp.python.mppy.mppy import Mpp

mpp = Mpp(project_name="spacetimeproject")

print("Project:", mpp.dm.PROJECT_ROOT_DIR)
print("MPP:", mpp.dm.MPP_ROOT_DIR)

In [ ]:
!conda env list
!conda install latexindent.pl -c conda-forge

In [ ]:
slices = [slice(0,3), slice(1,2), slice(1,2)]
def sliceup(lsls):
    res = []
    for ls, sli in zip(lsls, slices):
        res.extend(ls[sli])
    return res

def form_int(s):
    m = len(s)%3
    if m == 0:
        m = 3
    g = s[:m]
    for i in range(m,len(s)-3+1,3):
        g+= "\," + s[i: i+3]    
    return g

def format_line(line):
    elems = [e.strip() for e in line.split(",") if e.strip()]
    line_e = []
    for e in elems:
        if "." not in e and "," not in e:
            line_e.append(f(e))
        
    return " & ".join(line_e)

def form_scientific(s):
    try:
        s = str(int(s))
        return form_int(s)
    except: 
        pass
    e = 0
    s = float(s)
    while s < 1:
        s*=10
        e -= 1
    if e == 0:
        return "$" + str(s)[:5] + "$" 
    return "$" + str(s)[:5] + r"\mathrm{e}{" + str(e) + "}$" 

def form_fixed(s, p=5):
    #print("\n", "form_fixed", s)
    if type(s) is str:
        s = s.strip()
        if s.startswith("(") and s.endswith(")"):
            s = eval(s)
    
    if type(s) is tuple:
        #print("tup")
        return " $ " + r" \times ".join(form_int(str(k)) for k in s) + " $ "

    if type(s) in (int, str, float) and float(s) == int(float(s)):
        try:
            s = str(int(s))
            #print("int")
            return form_int(s)
        except: 
            pass
    s = float(s)
    z = f"{{s:.{p}f}}".format(s=s)
    
    if float(z) == 0:
        z+= "Precision too low."
        print("Precision too low.")
    #print("float")
    return f"$ {z} $"

for z in ["0.00123", "1.00123", "12345678910", ".00000001"]:
    print(form_scientific(z), "####", form_fixed(z))

In [ ]:
def parse(name):
    if "1" in name:
        t = "L2SpaceAtT1.000000_error"
        c0 = np.array([1, 1, 1])
    elif "2" in name:
        t = "L2SpaceAtT0.500000_error"
        c0 = np.array([1, 4, 2])
    
    mpp.reset_data()
    mpp.parse_log(os.getcwd()+"/notebooks/results/log_"+name+"_uniform")
    mpp.parse_log(os.getcwd()+"/notebooks/results/log_"+name+"_adaptive6")
    log_data= mpp.parse_log(os.getcwd()+"/notebooks/results/log_"+name+"_adaptive7")


    #print(log_data)
    keys = [        
        "meshsize",
        "Cells2", 
        "degree",
        "DoFs",    
        "hline",
        "||u_proj-u_h||_DG",
        "eta_(res,h)",
        "eta_(dg,h)",
        t, 
        "hline",
        "L2_Error",         
        "||u_conf - u  ||",
        "L2_proj_Error", 
        "eta_(conf,h)"                    
    ]
    data = {k:sliceup(v) for k,v in log_data.items() if k in keys}
    data["Cells2"] = [tuple(c0 * 2 ** level) for level in (6,7,8,6,7)]
    keys[1] = "Cells2"
    return keys, data

In [ ]:
key2tex = {
            "Cells" :                   r'\char"0023cells',
            "Cells2" :                  r'\char"0023cells',
            "DoFs":                     r'$\dim V_h$'      ,
            "L2_Error":                 r'$\big\|\vec u - \vec u_h\big\|_Q$',
            "L2_proj_Error":            r"$\big\|\vec u - \Pi_h\vec u\big\|_Q$",
            "||u_conf - u  ||":         r"$\big\|\vec u - \vec u_h^\text{\rm cf}\big\|_Q$",
            "L2SpaceAtT1.000000_error": r'$\big\|\vec u(T) - \vec u_h(T)\big\|_\Omega$',
            "L2SpaceAtT0.500000_error": r'$\big\|\vec u(T) - \vec u_h(T)\big\|_\Omega$',
            "||u_proj-u_h||_DG":        r"$\big\|\vec u - \vec u_h\big\|_{\DG,h}$",
            "eta_(res,h)":              r"$\eta_{\text{\rm res},h}$",
            "eta_(conf,h)":             r"$\eta_{{\mathrm L}_2,h}$",
            "eta_(dg,h)":               r"$\eta_{\DG,h}$ "
    }

In [ ]:


def table(keys, data):
    print(r'''\begin{table}[H]
\caption{Convergence test for ...} 
\centering
\begin{tabular}{l|ccc|cc}''')    
    for i, key in enumerate(keys):
        #print(i,k)
        if key == "meshsize":
            print('mesh size $h$    & $2^{-6}$    & $2^{-7}$     & $2^{-8}$  & $2^{-6}$ & $2^{-7}$')
            print(r"\\")
            continue
        if key == "degree":
            print(r'''$p_R= q_R$ & 1 & 1 & 1 & $0 \leq \cdot \leq 2$ & $0 \leq \cdot \leq 2$\\''')
            continue;
        if key == "hline":
            print(r"\hline")
            continue
            
        print(key2tex[key]," & ", end="")
        #print(data[key])
        print(" & ".join(form_fixed(g) for g in data[key]))
        if i < len(keys) - 1:
            print(r"\\")
        
    
    print(r"""\end{tabular}
\end{table}""")

In [ ]:
!ls notebooks/results

In [ ]:
table(*parse("exp1"))

In [ ]:
table(*parse("exp1_aligned"))

In [ ]:
table(*parse("exp2"))